In [ ]:
# import pickle

# training_data_path = "/scratch/data/open_teach/processed_data_pkl/grab_ball.pkl"

# with open(training_data_path, "rb") as f:
#     training_data = pickle.load(f)

# from pathlib import Path
# import imageio

# training_data.keys() # ['observations', 'max_cartesian', 'min_cartesian', 'max_gripper', 'min_gripper', 'task_emb']

# demo_num = 0

# max_cartesian = training_data['max_cartesian']
# min_cartesian = training_data['min_cartesian']
# max_gripper = training_data['max_gripper']
# min_gripper = training_data['min_gripper']

# print(f"max_cartesian: {max_cartesian}\nmin_cartesian: {min_cartesian}")
# len(training_data['observations']) # 100

# training_data['observations'][demo_num].keys() # ['demo_dir', 'pixels0', 'cartesian_states', 'gripper_states']

# cartesian_states = training_data['observations'][demo_num]['cartesian_states']
# gripper_states = training_data['observations'][demo_num]['gripper_states']

# frames = training_data['observations'][demo_num]['pixels0'][..., ::-1]
# video_path = Path(f"/home/danny/Downloads/demo_{demo_num}.mp4")
# video_path.parent.mkdir(parents=True, exist_ok=True)
# imageio.mimsave(video_path, frames, fps=30)

## Replay from Dataset sampled data

In [ ]:
# preprocess the actions
import pickle
from pathlib import Path
import imageio
from p3po.read_data.p3po_xarm import BCDataset, get_relative_action, get_quaternion_orientation

path = "/scratch/data/open_teach/processed_data_pkl"
processed_path = "./processed_data"
tasks = ["grab_ball"]
num_demos_per_task = 100
obs_type = "features"
history = True
history_len = 10
prompt =  "text"
temporal_agg = True
num_future_actions = 10
img_size = 128 # should not matter
action_after_steps = 1
intermediate_goal_step = 30
store_actions = True
training_keys = ["graph"]
subsample = 1
skip_first_n = 0
relative_actions = True

dataset = BCDataset(
    path=path,
    processed_path=processed_path,
    tasks=tasks,
    num_demos_per_task=num_demos_per_task,
    obs_type=obs_type,
    history=history,
    history_len=history_len,
    prompt=prompt,
    temporal_agg=temporal_agg,
    num_future_actions=num_future_actions,
    img_size=img_size,
    action_after_steps=action_after_steps,
    intermediate_goal_step=intermediate_goal_step,
    store_actions=store_actions,
    training_keys=training_keys,
    subsample=subsample,
    skip_first_n=skip_first_n,
    relative_actions=relative_actions,
    use_quaternion_orientation=True
)

Loading /scratch/data/open_teach/processed_data_pkl/grab_ball.pkl


In [ ]:
from dm_env import specs
import pickle
import numpy as np
from p3po.replay_buffer import make_expert_replay_loader

action_spec = specs.BoundedArray(
    (dataset._max_action_dim,),
    np.float32,
    dataset.stats["actions"]["min"],
    dataset.stats["actions"]["max"],
    "action",
)

batch_size=1 # actual training used 64
expert_replay_loader = make_expert_replay_loader(dataset, batch_size=batch_size)
sample = next(iter(expert_replay_loader))
stats = expert_replay_loader.dataset.stats # stats passed to agent act function

Value(False)


ERROR:tornado.general:SEND Error: Host unreachable


In [ ]:
actions = sample["actions"]
proprioceptive = sample["proprioceptive"]
graph = sample["graph"]

# shape: B x history_len x? x action_dim
actions[0]

torch.Size([1, 10, 10, 7])

In [ ]:
import numpy as np
import torch

task_idx = 0
_keys = dataset._keys
_history_len = dataset._history_len

episodes = dataset._episodes[task_idx]
n_episodes = len(episodes)
print(f"Length of episodes: {n_episodes}")
episode_one = episodes[0]
print(episode_one.keys())

action = episode_one["action"] # 170x7
observations = episode_one["observation"]
gripper_states = episode_one["observation"]["gripper_states"] # 170
cartesian_states = episode_one["observation"]["cartesian_states"] #  170x7
graph = episode_one["observation"]["graph"] # 170x60

actions = episode_one["action"]

task_emb = 0

sample_idx = np.random.randint(
    0, len(observations[_keys[0]]) - _history_len
)
sampled_input = {}


sampled_input = {}
for key in _keys:
    sampled_input[key] = observations[key][
        sample_idx : sample_idx + dataset._history_len
    ]

start_idx = sample_idx
end_idx = sample_idx + _history_len

# combine cartesian states and gripper states
cartesian_states = observations["cartesian_states"][start_idx : end_idx]
gripper_states = observations["gripper_states"][start_idx : end_idx][:, None]

sampled_proprioceptive_state = np.concatenate(
    [cartesian_states, gripper_states],
    axis=1,
)

if dataset._temporal_agg:
    # arrange sampled action to be of shape (history_len, num_queries, action_dim)
    sampled_action = np.zeros(
        (dataset._history_len, dataset._num_future_actions, actions.shape[-1])
    )

    # -1 since its num_queries including the last action of the history
    # the target number of actions we want, but we might not be able to get
    n_actions_needed = (dataset._history_len + dataset._num_future_actions - 1)
    act = np.zeros((n_actions_needed, actions.shape[-1]))

    # extract n_actions_needed actions starting at start idx.
    avail_actions = min(len(actions), start_idx + n_actions_needed)
    act[: avail_actions - start_idx] = actions[
        start_idx : start_idx + n_actions_needed
    ]

    # pad with last action if we didn't have enough
    if len(actions) < start_idx + n_actions_needed:
        act[len(actions) - start_idx :] = actions[-1]

    # overlap actions
    sampled_action = np.lib.stride_tricks.sliding_window_view(
        act, (dataset._num_future_actions, actions.shape[-1])
    )
    sampled_action = sampled_action[:, 0]
else:
    sampled_action = actions[start_idx : start_idx + dataset._history_len]

return_dict = {}
for key in dataset._keys:
    return_dict[key] = sampled_input[key]
return_dict["proprioceptive"] = dataset.preprocess["proprioceptive"](
    sampled_proprioceptive_state
)
return_dict["actions"] = dataset.preprocess["actions"](sampled_action)
return_dict["task_emb"] = task_emb

Length of episodes: 65
dict_keys(['observation', 'action', 'task_emb'])
Value(False)


In [ ]:
import numpy as np

x = np.arange(19*7).reshape((19, 7))

# x is: [0 1 2 3 4 5]


# Create a sliding window view with a window size of 3
(num_future_actions, actions.shape[-1]) # 10 x 7
v = np.lib.stride_tricks.sliding_window_view(x, (num_future_actions, actions.shape[-1]))
# v is:
# [[0 1 2]
#  [1 2 3]
#  [2 3 4]
#  [3 4 5]]

# Calculate the moving average
# moving_average = v.mean(axis=-1)
# # moving_average is: [1. 2. 3. 4.]
# print(v)
# print(moving_average)

v.shape

(10, 1, 10, 7)